In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
from os import listdir
from os.path import isfile, isdir, join


In [2]:
def readImg(src):
    head = cv2.imread(src[0], cv2.IMREAD_GRAYSCALE)
    img = np.zeros((head.shape[0], head.shape[1], 4), dtype='uint8')

    img[:,:,0] = head
    img[:,:,1] = cv2.imread(src[1], cv2.IMREAD_GRAYSCALE)
    img[:,:,2] = cv2.imread(src[2], cv2.IMREAD_GRAYSCALE)
    img[:,:,3] = cv2.imread(src[3], cv2.IMREAD_GRAYSCALE)

    return img

In [20]:
# Brute-Force Matching with SIFT Descriptors

def SIFT(img):
    siftDetector= cv2.xfeatures2d.SIFT_create(2000)
    kp, des = siftDetector.detectAndCompute(img, None)
    return kp, des

def matcher(des1, des2): 
    # create Matcher object
    bf_matcher = cv2.BFMatcher()

    # Match descriptors.
    matches = bf_matcher.match(des1, des2, None)  #Creates a list of all matches, just like keypoints

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x: x.distance)
    return matches

def getHomography(matches, kp1, kp2):
    points1 = np.zeros((len(matches), 2), dtype=np.float32)  #Prints empty array of size equal to (matches, 2)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = kp1[match.queryIdx].pt    #gives index of the descriptor in the list of query descriptors
        points2[i, :] = kp2[match.trainIdx].pt    #gives index of the descriptor in the list of train descriptors

    #h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)
    h, mask = cv2.estimateAffinePartial2D(points2, points1)
    return h

def register(img):
    for i in [1,2,3]:
        # Find matching points
        kp1, des1 = SIFT(img[:,:,0])
        kp2, des2 = SIFT(img[:,:,i])
        matches = matcher(des1, des2)

        # Use homography
        h = getHomography(matches, kp1, kp2)
        height, width, channels = img.shape
        #img[:,:,i] = cv2.warpPerspective(img[:,:,i], h, (width, height))  #Applies a perspective transformation to an image.
        img[:,:,i] = cv2.warpAffine(img[:,:,i], h, (width, height))

        # print("Estimated homography : \n",  h)
    return img

In [21]:
def saveImg(img, f, tPath):
    fullpath = join(tPath, f)
    os.mkdir(fullpath)
    RGBpath = join(fullpath, 'RGB.tif')
    FalseColorpath = join(fullpath, 'FalseColor.tif')
    cv2.imwrite(RGBpath, img[:, :, :3])
    cv2.imwrite(FalseColorpath, img[:, :, 1:4])
    print('Image "{}" saved'.format(f))
    
def process(f): 
    dataPath = 'C:/Users/tmyda/Documents/UAV_color'
    targetPath = 'C:/Users/tmyda/Documents/SIFT'
    
    fullpath = join(dataPath, f)
    if isfile(fullpath): pass
    elif isdir(fullpath):
        srcImg = listdir(fullpath)
        for index, item in enumerate(srcImg):
            srcImg[index] = join(fullpath, item)
        img = readImg(srcImg)
        register(img)
        saveImg(img, f, targetPath)
    
    return ''

In [ ]:
if __name__ == '__main__':
    dataPath = 'C:/Users/tmyda/Documents/UAV_color'
    targetPath = 'C:/Users/tmyda/Documents/SIFT'
    os.mkdir(targetPath)
    files = listdir(dataPath)
    
    for f in files:
        process(f)


In [22]:
##### Produce Training Data #####
# 1. Homography matrix
# 2. Descripters of ground truth and NIR images

def getNIRFilePath(fPath):
    files = listdir(fPath)
    NirPath = []
    RegPath = []
    for name in files:
        f = join(fPath, name)
        if isfile(f): pass
        elif isdir(f):
            img1Path = join(f, 'IMG_{name}_4.tif'.format(name=name))
            img2Path = join(f, 'IMG_{name}_4toIMG_{name}_2_registered.tif'.format(name=name))
            NirPath.append(img1Path)
            RegPath.append(img2Path)
 
    return NirPath, RegPath

def getGroundTruthFilePath(fPath):
    files = listdir(fPath)
    groundPath = []
    for name in files:
        f = join(fPath, name)
        if isfile(f): pass
        elif isdir(f):
            imgPath = join(f, 'IMG_{name}_2.tif'.format(name=name))
            groundPath.append(imgPath)
 
    return groundPath

def getHomographyMat(src, trg):
    Homos = []
    for i in range(len(src)):
        # Find matching points
        srcImg = cv2.imread(src[i], cv2.IMREAD_GRAYSCALE)
        trgImg = cv2.imread(trg[i], cv2.IMREAD_GRAYSCALE) 
        
        kp1, des1 = SIFT(trgImg)
        kp2, des2 = SIFT(srcImg)
        matches = matcher(des1, des2)

        # Use homography
        h = getHomography(matches, kp1, kp2)
        Homos.append(h)
    Homos = np.stack(Homos, axis=0)
    
    return Homos

def getDescripters(path):
    Des = []
    for i in range(len(path)):
        img = cv2.imread(path[i], cv2.IMREAD_GRAYSCALE)
        kp, des = SIFT(img)
        
        if des.shape[0] < 2000:
            diff = 2000 - des.shape[1]
            des = np.pad(des, ((0, diff), (0, 0)), 'constant')
            
        des = np.swapaxes(des[:2000, :], 0, 1)
        Des.append(des)
        
    Des = np.stack(Des, axis=0)
    return Des
    

In [23]:
# Homography data

path = 'C:/Users/tmyda/Documents/UAV_reg/Reg'
Nir, Reg = getNIRFilePath(path)

H = getHomographyMat(Nir, Reg)
np.save('HomoMat', H)
print('Save numpy array, shape: {}'.format(H.shape))

# Descripters data
path = 'C:/Users/tmyda/Documents/UAV_reg/Reg'
Ground = getGroundTruthFilePath(path)
NirDes = getDescripters(Nir) / 512
GroundDes = getDescripters(Ground) / 512


np.save('NirDes', NirDes)
np.save('GroundDes', GroundDes)
print('Save numpy array, shape: {}'.format(GroundDes.shape))
print('Save numpy array, shape: {}'.format(NirDes.shape))

Save numpy array, shape: (137, 2, 3)
Save numpy array, shape: (137, 128, 2000)
Save numpy array, shape: (137, 128, 2000)


In [6]:
H = np.load('HomoMat.npy')

name = '0168'

head = cv2.imread('C:/Users/tmyda/Documents/UAV_reg/Reg/{name}/IMG_{name}_2.tif'.format(name=name), cv2.IMREAD_GRAYSCALE)
img = np.zeros((head.shape[0], head.shape[1], 3), dtype='uint8')
img[:, :, 0] = head
img[:, :, 1] = cv2.imread('C:/Users/tmyda/Documents/UAV_reg/Reg/{name}/IMG_{name}_3toIMG_{name}_2_registered.tif'.format(name=name), cv2.IMREAD_GRAYSCALE)
img[:, :, 2] = cv2.imread('C:/Users/tmyda/Documents/UAV_reg/Reg/{name}/IMG_{name}_4.tif'.format(name=name), cv2.IMREAD_GRAYSCALE)

h = H[:, :, 1]
height, width, channels = img.shape
img[:,:,2] = cv2.warpPerspective(img[:,:,2], h, (width, height))
cv2.imshow('Reg', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [78]:
path1 = 'C:/Users/tmyda/Documents/UAV_reg/Reg'
path2 = 'C:/Users/tmyda/Documents/SIFT_partialAffine'

file1 = listdir(path1)
file2 = listdir(path2)

NotMatch = set(file2) - set(file1)


In [5]:

img = cv2.imread('C:/Users/tmyda/Documents/UAV_reg/Reg/{name}/IMG_{name}_2.tif'.format(name=name), cv2.IMREAD_GRAYSCALE)
img.shape

(960, 1280)